#### Q1. Running Elastic

In [1]:
!curl localhost:9200

{
  "name" : "2b31128447d1",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "vHEF4eCZTyKe8D2BF4F72w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


#### Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
!pip install requests

#### Q2.Indexing the data

##### The function used to index data in elastic search is `index()`

#### Q3.Searching

##### Defining a query

In [4]:
query = "How do I execute a command in a running docker container?"

##### Defining the search query

In [11]:
search_query = {
    "size":5,
    "query":{
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            
        }
    }
    
}

##### Searching

In [12]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
index_name = "course-questions"

In [13]:
response = es_client.search(index=index_name, body=search_query)

In [15]:
response['hits']['hits'][0]['_score']

84.050095

In [17]:
search_query = {
    "size":3,
    "query":{
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter":{
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
            
        }
    }
    
}

In [18]:
response2 = es_client.search(index=index_name, body=search_query)

In [23]:
response2['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [28]:
response2['hits']['hits'][2]['_source']['text']

'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan'

#### Q5. Building a prompt

In [24]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [41]:
questions = [quest['_source']['question'] for quest in response2['hits']['hits']]
answers = [ans['_source']['text'] for ans in response2['hits']['hits']]
context = []
for i in range(len(questions)):
    temp = context_template.format(question=questions[i], text=answers[i])
    context.append(temp)

In [66]:
context

['Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 "Q: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
 'Q: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comm

In [42]:
contexts = "\n\n".join(context)
print(contexts)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

##### Finalizing prompt

In [43]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [46]:
final_prompt = prompt_template.format(question=query, context=contexts)
print(final_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

##### Length of the final prompt

In [47]:
len(final_prompt)

1462

#### Q6.Tokens

In [48]:
!pip install tiktoken

  Using cached tiktoken-0.7.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached regex-2024.5.15-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
Using cached tiktoken-0.7.0-cp311-cp311-macosx_11_0_arm64.whl (907 kB)
Using cached regex-2024.5.15-cp311-cp311-macosx_11_0_arm64.whl (278 kB)


In [50]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

##### Number of tokens for the final prompt

In [52]:
len(encoding.encode(final_prompt))

322

In [53]:
encoding.decode_single_token_bytes(63842)

b"You're"

#### Bonus generating the answer

In [57]:
from openai import OpenAI
import os
from dotenv import load_dotenv

In [58]:
load_dotenv()

True

In [59]:
client = OpenAI()

In [60]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "user",
         "content": f"{final_prompt}"
        }
    ]
)

In [65]:
for choice in response.choices:
    print(choice.message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. Here is how you can do it:

1. First, find the container ID of the running container by using the `docker ps` command:
   ```bash
   docker ps
   ```

2. Then, execute the desired command in the container. For example, to start a bash session, you would use:
   ```bash
   docker exec -it <container-id> bash
   ```
